In [1]:
import pandas as pd
import requests
from tqdm import tqdm
import random

# Useful Links
Implementing data object for neural networks:
https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html

Writing a custom dataset class: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

Writing a GAT Class example: https://github.com/pyg-team/pytorch_geometric/blob/master/examples/gat.py

In [2]:
scRNA_data = pd.read_csv('GSE200981_scRNAseq_processed.tsv', sep='\t')
scRNA_data.index = scRNA_data['Gene.names']
scRNA_data = scRNA_data.drop('Gene.names', axis=1)
len(scRNA_data)

26364

In [3]:
#Mapping string to protein names
string_api_url = "https://string-db.org/api"
output_format = "tsv-no-header"
method = "get_string_ids"

params = {

    "identifiers" : "\r".join(list(scRNA_data.index)), # your protein list
    "limit": 1,
    "echo_query": 1,
    "species" : 9606, # species NCBI identifier 
    "caller_identity" : "www.awesome_app.org" # your app name

}

request_url = "/".join([string_api_url, output_format, method])

results = requests.post(request_url, data=params)


protein_2_string = dict()
string_2_protein = dict()

for line in results.text.strip().split("\n"):
    l = line.split("\t")
    protein_identifier, string_identifier = l[0], l[2]
    protein_2_string[protein_identifier] = string_identifier
    string_2_protein[string_identifier] = protein_identifier

In [4]:
scRNA_data = scRNA_data.loc[list(protein_2_string.keys())]
scRNA_data

,V1_T0,V2_T0,V3_T0,V4_T0,V5_T0,V6_T0,V7_T0,V8_T0,V9_T0,V10_T0,...,V247_T7,V248_T7,V249_T7,V250_T7,V251_T7,V252_T7,V253_T7,V254_T7,V255_T7,V256_T7
Gene.names,,,,,,,,,,,,,,,,,,,,,
OR4F5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OR4F3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OR4F29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
OR4F16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SAMD11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DAZ1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DAZ3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DAZ2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#Getting network pairs from interaction file
file = open('9606.protein.links.v12.0.txt', 'r')
lines = file.readlines()
lines.pop(0)

network_pairs = set()

for line in tqdm(lines):
    line = line.strip().split(' ')
    #print(line)
    
    if int(line[2]) >= 700:
    
        try:
            p1 = string_2_protein[line[0]]
            p2 = string_2_protein[line[1]]
            
            network_pairs.add((p1, p2))
        
        except KeyError:
            continue
        

100%|██████████| 13715404/13715404 [00:10<00:00, 1296940.03it/s]


In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch.utils.data import Dataset
from torch_geometric.nn import GATConv
import torch.nn as nn

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super().__init__()
        
        self.conv1 = GATConv(in_channels, out_channels, heads, dropout = 0.6)
        self.output_layer = nn.Linear(hidden_channels*out_channels, out_channels)
        #self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False, dropout=0.6)
        #print(self.conv1.weight.dtype)
        
    def forward(self, x, edge_index):
        #x = F.dropout(x, p=0.6)
        x = self.conv1(x, edge_index)
        #print(x.size())
        x = torch.flatten(x)
        x = self.output_layer(x)
        #print(x.size())
        x = F.softmax(x)
        #x = F.dropout(x, p=0.6)
        #x = self.conv2(x, edge_index)
        return x
        

In [8]:
filename = '9606.protein.links.v12.0.txt'

class EMT_Dataset(Dataset):
    def __init__(self, filename):
        #self.idx = 0
        
        file = open(filename, 'r')
        lines = file.readlines()
        lines.pop(0)
    
        string_2_index = dict()
        counter = 0
        for string_id in string_2_protein:
            string_2_index[string_id] = counter
            counter += 1
            
        list_network = list()
        self.node_features = list()
        self.list_outputs = list()
    
        print('Getting network tensor...')
        for line in tqdm(lines):
            line = line.strip().split(' ')
    
            if int(line[2]) >= 999:
    
                try:
                    id1 = string_2_index[line[0]]
                    id2 = string_2_index[line[1]]
                    list_network.append([id1, id2])
                    list_network.append([id2, id1])
        
                except KeyError:
                    continue
    
        print('Getting node features tensor...')
        T0_column_vals = [column for column in scRNA_data.columns if 'T0' in column]
        T8_column_vals = [column for column in scRNA_data.columns if 'T7' in column]
        #print(T8_column_vals)
        proteins = [string_2_protein[string_id] for string_id in string_2_index]
        
        for column in T0_column_vals:
            self.node_features.append([[scRNA_data.loc[protein, column]] for protein in proteins])
            self.list_outputs.append([1,0])
        #print(self.node_features[0])
        for column in T8_column_vals:
            #print('Hello')
            self.node_features.append([[scRNA_data.loc[protein, column]] for protein in proteins])
            self.list_outputs.append([0,1])
        
        #print([0, 1] in self.list_outputs)
        #print(len(self.node_features[0]))
        self.edge_index = torch.tensor(list_network).t().contiguous()
        self.node_features = torch.tensor(self.node_features, dtype=torch.float)
        self.list_outputs = torch.tensor(self.list_outputs, dtype=torch.float)                            
    
    def __getitem__(self, idx):
        graph = self.edge_index
        node_feature = torch.transpose(self.node_features[idx], 0, 1).t()
        output = self.list_outputs[idx]
        
        #self.idx += 1
        #if self.idx == len(self.node_features):
            #self.idx = 0
        
        return graph, node_feature, output
    
    def __len__(self):
        return len(self.node_features)
    
    def num_features(self):
        return self.node_features[0].shape[0]
    
    def num_classes(self):
        return self.list_outputs[0].shape[0]
    
    def shuffle(self):
        temp = list(zip(self.node_features, self.list_outputs))
        random.shuffle(temp)
        self.node_features, self.list_outputs = zip(*temp)
        self.node_features, self.list_outputs = list(self.node_features), list(self.list_outputs)

In [9]:
dataset = EMT_Dataset(filename)

  1%|          | 151664/13715404 [00:00<00:08, 1516507.67it/s]

Getting network tensor...


100%|██████████| 13715404/13715404 [00:08<00:00, 1567171.13it/s]


Getting node features tensor...


In [10]:
dataset.shuffle()

In [23]:
def train(model, dataset, epochs, lr = 1e-8, weight_decay = 5e-4):
    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss().to(device)
    model.train()
    optimizer.zero_grad()
    
    for _ in range(epochs):
        model = model.train()
        
        for i in range(len(dataset)):
            optimizer.zero_grad()
            
            graph, node_features, output = dataset.__getitem__(i)
            graph = graph.to(device)
            node_features = node_features.to(device)
    
            out = model(node_features, graph)
            print(out, output)
            loss = criterion(out, output)
            loss.backward()
            optimizer.step()
            print(float(loss))

In [24]:
model = GAT(1, dataset.num_features(), dataset.num_classes(), 1).to(device)

In [25]:
dataset.num_features(), dataset.num_classes()

(18840, 2)

In [26]:
train(model, dataset, 1000)

<ipython-input-7-89fd749cb75c>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


tensor([0.9575, 0.0425], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.0018046649638563395
tensor([0.6596, 0.3404], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.43507251143455505
tensor([0.9414, 0.0586], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.0034291474148631096
tensor([9.9941e-01, 5.9134e-04], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
3.496461999930034e-07
tensor([0.4308, 0.5692], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.18557988107204437
tensor([0.4718, 0.5282], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.2226019650697708
tensor([0.4694, 0.5306], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.28156009316444397
tensor([0.0139, 0.9861], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.0001934048777911812
tensor([0.9948, 0.0052], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9895423054695129
tensor([0.1692, 0.8308], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.028622418642044067
tensor([0.9859, 0.0141], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.00019936580793000758
ten

0.3448559641838074
tensor([9.9996e-01, 4.0978e-05], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.6804342362064517e-09
tensor([1.0000e+00, 1.2405e-06], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.4799910429130758e-12
tensor([0.5054, 0.4946], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.2554698884487152
tensor([0.6070, 0.3930], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.36848875880241394
tensor([0.9979, 0.0021], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9958581328392029
tensor([0.9914, 0.0086], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9827816486358643
tensor([9.9975e-01, 2.5091e-04], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9994982481002808
tensor([1.0000e+00, 1.0805e-11], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
5.837505696496897e-23
tensor([9.9996e-01, 4.3473e-05], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.891567125156257e-09
tensor([9.9985e-01, 1.4680e-04], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9997063279151917
tensor([2.0994e-04, 9.9979e-01], grad_

0.005489550065249205
tensor([0.0933, 0.9067], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.8220751881599426
tensor([0.9311, 0.0689], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.004745482467114925
tensor([0.2389, 0.7611], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.5793395638465881
tensor([1.0000e+00, 1.4299e-10], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.022274148859857e-20
tensor([0.9860, 0.0140], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9721803069114685
tensor([0.7894, 0.2106], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.04437117278575897
tensor([9.9994e-01, 6.4729e-05], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
4.1899674840806256e-09
tensor([0.9956, 0.0044], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.9736015019589104e-05
tensor([9.9999e-01, 6.1682e-06], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
3.823655975176976e-11
tensor([0.9977, 0.0023], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
5.225679615250556e-06
tensor([0.0070, 0.9930], grad_fn=<SoftmaxBackward>) ten

0.3142705261707306
tensor([0.9697, 0.0303], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9402927756309509
tensor([0.9989, 0.0011], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.1711274510162184e-06
tensor([9.9997e-01, 3.3075e-05], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9999338984489441
tensor([0.4099, 0.5901], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.1680075228214264
tensor([0.9680, 0.0320], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9369867444038391
tensor([9.9950e-01, 5.0001e-04], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
2.49987010647601e-07
tensor([0.7724, 0.2276], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.05181696638464928
tensor([9.9986e-01, 1.3957e-04], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.9482818913729716e-08
tensor([0.8313, 0.1687], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.028443222865462303
tensor([0.9285, 0.0715], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.005115913227200508
tensor([0.1392, 0.8608], grad_fn=<SoftmaxBackward>) tensor([0.

...

0.6716874837875366
tensor([6.7684e-05, 9.9993e-01], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.9998645782470703
tensor([0.4579, 0.5421], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.20968294143676758
tensor([0.9633, 0.0367], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9278771281242371
tensor([9.9998e-01, 1.6360e-05], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9999673366546631
tensor([0.9848, 0.0152], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.00023095984943211079
tensor([0.9786, 0.0214], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.00045925681479275227
tensor([0.0111, 0.9889], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.00012414137017913163
tensor([0.5572, 0.4428], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.3104221820831299
tensor([0.3929, 0.6071], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.15435221791267395
tensor([0.9988, 0.0012], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.5427124253619695e-06
tensor([0.6278, 0.3722], grad_fn=<SoftmaxBackward>) tensor([0., 1.]

3.3501321013318375e-05
tensor([9.9999e-01, 5.8143e-06], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
3.396291581303501e-11
tensor([0.8295, 0.1705], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.6880912780761719
tensor([0.4710, 0.5290], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.22184841334819794
tensor([0.5549, 0.4451], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.30795586109161377
tensor([0.4641, 0.5359], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.2872389554977417
tensor([0.0352, 0.9648], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.0012363515561446548
tensor([1.0000e+00, 1.0504e-06], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.1272139905510459e-12
tensor([0.2912, 0.7088], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.08480208367109299
tensor([1.0000e+00, 7.2732e-07], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9999985694885254
tensor([0.6164, 0.3836], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.37998560070991516
tensor([1.0000e+00, 1.0891e-06], grad_fn=<SoftmaxBackward>)

5.626245996870338e-11
tensor([0.0183, 0.9817], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.0003358784597367048
tensor([0.2631, 0.7369], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.06922093033790588
tensor([0.3699, 0.6301], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.3970149755477905
tensor([9.9965e-01, 3.5346e-04], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.2493234180510626e-07
tensor([1.0000e+00, 2.1400e-06], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9999957084655762
tensor([0.0058, 0.9942], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
3.364326403243467e-05
tensor([9.9976e-01, 2.4062e-04], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
5.7899967487173853e-08
tensor([0.9984, 0.0016], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
2.4525963908672566e-06
tensor([9.9968e-01, 3.2144e-04], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.0330622046694771e-07
tensor([9.9954e-01, 4.5731e-04], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
2.091226463107887e-07
tensor([0.0108, 0.9892], grad_fn

0.9999997019767761
tensor([0.9366, 0.0634], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
0.004018144682049751
tensor([9.9987e-01, 1.3122e-04], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.7222776094172332e-08
tensor([0.8461, 0.1539], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.7158709168434143
tensor([1.0000e+00, 1.0406e-06], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
1.1169628590101954e-12
tensor([0.1690, 0.8310], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.028573039919137955
tensor([9.9921e-01, 7.8896e-04], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
6.2252763655124e-07
tensor([9.9936e-01, 6.4182e-04], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.9987168312072754
tensor([0.5239, 0.4761], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.2745009660720825
tensor([0.7601, 0.2399], grad_fn=<SoftmaxBackward>) tensor([0., 1.])
0.5777149796485901
tensor([0.9936, 0.0064], grad_fn=<SoftmaxBackward>) tensor([1., 0.])
4.0945033106254414e-05
tensor([0.9822, 0.0178], grad_fn=<SoftmaxBackward>) 

KeyboardInterrupt: 

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
edge_index.size()

In [ ]:
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
x.size()

In [ ]:
x

In [ ]:
dataset.num_features(), dataset.num_classes()

In [ ]:
sum(p.numel() for p in model.parameters())